In [1]:
from lib.utils import load_raw_by_path
eeg = load_raw_by_path(f'22-AGING-10.edf').get_data(picks='EEG')[0]
import torch
X = torch.from_numpy(eeg.reshape(-1,1,5000)).float()
from lib.models import CNN_0
model = CNN_0()
model.load_state_dict(torch.load(f='model_pt_epoch_1000.pt'))
from torch.nn.functional import softmax
y_pred = softmax(model(X),dim=1).argmax(axis=1)
import pandas as pd
df = pd.DataFrame(y_pred)
df[df[0] == 2] = 'W'
df[df[0] == 1] = 'S'
df[df[0] == 0] = 'P'
df.to_csv('pred.csv',index=False)

/home/andrew/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Parameter containing:
tensor([0.1000, 1.0000, 1.0000], requires_grad=True)


In [2]:
import sqlite3
from sqlite3 import Error
import os
offset = 10e7       #epoch time period
rename_dict = {'W':'Sleep-Wake', 'S':'Sleep-SWS', 'P':'Sleep-Paradoxical', 'X':''}

csv_filename = f'pred.csv'
zdb_filename = f'22-AGING-10.zdb'

df = pd.read_csv(csv_filename)
try:
    conn = sqlite3.connect(zdb_filename)
except Error as e:
    print(e)
cur = conn.cursor()

#drop this table - creates issues
query = "DROP TABLE IF EXISTS temporary_scoring_marker;"
cur.execute(query)

#get keyid of scoring
query = "SELECT MAX(id) FROM scoring_revision WHERE name='Machine Data'"
cur.execute(query)
keyid = cur.fetchall()[0][0]

#get starting point for scoring
query = "SELECT id FROM scoring_marker WHERE type LIKE 'Sleep%' AND key_id='"+str(keyid)+"';"
cur.execute(query)
startid = cur.fetchall()[0][0]

#get start time to crreate epochs
query = 'SELECT starts_at FROM scoring_marker WHERE id = '+str(startid)+";"
cur.execute(query)
start_time = cur.fetchall()[0][0]
stop_time = 0

#delete first score before adding machine data
query = "DELETE FROM scoring_marker;"
cur.execute(query)

#insert new epochs with scoring into the table
for i in range(len(df)):
    #calculate epoch
    if i != 0:
        start_time = stop_time
    stop_time = start_time+offset

    score = rename_dict[df.at[i,'0']]
    #insert epoch
    query = f"""
            INSERT INTO scoring_marker 
            (starts_at, ends_at, notes, type, location, is_deleted, key_id)
            VALUES 
            ({start_time}, {stop_time}, '', '{score}', '', 0, {keyid});
            """ 
    cur.execute(query)

conn.commit()
conn.close()